Let's see if we can't beat the sample-based solution (for the first half of this problem, at least)

In [10]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTROL_DT, PITCH_DT, CONTACT_DT, NUM_JOINTS
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS
from iiwa_batter.physics import (
    PITCH_START_POSITION,
    FLIGHT_TIME_MULTIPLE,
    find_ball_initial_velocity,
)
from iiwa_batter.trajectory_library import LIBRARY_POSITIONS, LIBRARY_SPEEDS_MPH

from iiwa_batter.swing_simulator import setup_simulator, run_swing_simulation, reset_systems
from iiwa_batter.swing_optimization.stochastic_gradient_descent import make_torque_trajectory, make_trajectory_timesteps
from iiwa_batter.swing_optimization.full_trajectory import full_trajectory_reward
from iiwa_batter.swing_optimization.swing_impact import calculate_plate_time_and_ball_state, dummy_torque_trajectory
from iiwa_batter.swing_optimization.partial_trajectory import partial_trajectory_reward
from iiwa_batter.swing_optimization.graduate_student_descent import student_control_vector, keyframe_trajectory, trajectory_to_control_vector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Swing Impact Optimization

In [11]:
meshcat = StartMeshcat()
robot = "iiwa14"
robot_constraints = JOINT_CONSTRAINTS[robot]
target_speed_mph = LIBRARY_SPEEDS_MPH[0]
target_position = LIBRARY_POSITIONS[0]
plate_time, plate_ball_position, plate_ball_velocity = calculate_plate_time_and_ball_state(target_speed_mph, target_position)
simulator, diagram = setup_simulator(torque_trajectory={}, model_urdf=robot, dt=CONTACT_DT, robot_constraints=robot_constraints, meshcat=meshcat)

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [12]:
plate_joint_position_controls = np.array([0.75, 0.25, 0.01, -0.6, 0, 0.55, 0])
plate_joint_velocity_controls = np.array([1, 0, 1, 0, -0.4, 0, -0.5])

# Swing impact optimization, see what the swing looks like
torque_trajectory = dummy_torque_trajectory(plate_time)
reset_systems(diagram, torque_trajectory)
status_dict = run_swing_simulation(
    simulator,
    diagram,
    start_time=plate_time,
    end_time=max(torque_trajectory.keys()),
    initial_joint_positions=student_control_vector(robot, plate_joint_position_controls, "position"),
    initial_joint_velocities=student_control_vector(robot, plate_joint_velocity_controls, "velocity"),
    initial_ball_position=plate_ball_position,
    initial_ball_velocity=plate_ball_velocity,
    meshcat=meshcat,
)


In [13]:
plate_joint_position_controls = np.array([0.75, 0.25, 0.01, -0.6, 0, 0.55, 0])
plate_joint_velocity_controls = np.array([1, 0, 1, 0, -0.4, 0, -0.5])

# Swing impact optimization, get the actual reward
present_reward = partial_trajectory_reward(
    simulator=simulator,
    diagram=diagram,
    start_time=plate_time,
    initial_joint_positions=student_control_vector(robot, plate_joint_position_controls, "position"),
    initial_joint_velocities=student_control_vector(robot, plate_joint_velocity_controls, "velocity"),
    initial_ball_position=plate_ball_position,
    initial_ball_velocity=plate_ball_velocity,
    torque_trajectory=dummy_torque_trajectory(plate_time),
)

print(present_reward)

30.215659375981474


## Link Trajectory Optimization

In [6]:
simulator, diagram = setup_simulator(torque_trajectory={}, model_urdf=robot, dt=PITCH_DT, robot_constraints=robot_constraints, meshcat=meshcat)
pitch_velocity, ball_time_of_flight = find_ball_initial_velocity(target_speed_mph, target_position)
trajectory_timesteps = make_trajectory_timesteps(ball_time_of_flight)

In [9]:
# Link trajectory optimization, see what the swing looks like
initial_joint_position_controls = np.array([0.5, 0.25, 0.01, -0.6, 0, 0.55, 0])
torque_keyframe_controls = {
    0: np.array([1, -0.2, 0, 0.2, 0, 0, -0.5]),
    ball_time_of_flight: np.zeros(NUM_JOINTS),
}
torque_trajectory = keyframe_trajectory(robot, trajectory_timesteps, torque_keyframe_controls)
trajectory_to_control_vector(robot, torque_trajectory)
reset_systems(diagram, torque_trajectory)
status_dict = run_swing_simulation(
    simulator,
    diagram,
    start_time=0,
    end_time=ball_time_of_flight+0.1,
    initial_joint_positions=student_control_vector(robot, initial_joint_position_controls, "position"),
    initial_joint_velocities=np.zeros(NUM_JOINTS),
    initial_ball_position=PITCH_START_POSITION,
    initial_ball_velocity=pitch_velocity,
    meshcat=meshcat,
)